In [2]:
import math

import numpy
import pandas

import ipywidgets as widgets
from IPython.display import display, clear_output

import gensim
from gensim.corpora import Dictionary
from gensim.models import TfidfModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

import pythainlp

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans

from data_tokenizer import load_corpus

from model.new_sdc import NewSDC
from model.sdc import SDC

### Load Data

In [3]:
file_name = 'TescoLotus'

corpus = load_corpus('../data/' + file_name + '.txt')

print('Total documents', len(corpus))

f = open('../data/tokenized/sentencepiece/tokenized_' + file_name + '.txt')
tokenized_corpus = eval(f.read())
f.close()

Total documents 268


### Preprocess Corpus

#### Remove Words

In [4]:
dictionary = Dictionary(tokenized_corpus)
print('origin:', len(dictionary), 'words')

dictionary.filter_extremes(no_below=2, no_above=0.7, keep_n=len(dictionary))
print('filter frequent words:', len(dictionary), 'words')

letter_words = [id for id in range(len(dictionary)) if len(dictionary[id]) <= 1] 
dictionary.filter_tokens(bad_ids=letter_words)
print('filter letter words:', len(dictionary), 'words')

stopwords = pythainlp.corpus.stopwords.words('thai')
stopwords.extend(['นี้'])
dictionary.add_documents([stopwords])
stopwords = [dictionary.token2id[word] for word in stopwords]
dictionary.filter_tokens(bad_ids=stopwords)
print('filter stop words:', len(dictionary), 'words')

origin: 962 words
filter frequent words: 906 words
filter letter words: 847 words
filter stop words: 651 words


In [5]:
idx_corpus = [dictionary.doc2idx(doc) for doc in tokenized_corpus]

temp_corpus = []
for doc in idx_corpus:
    temp_corpus.append([dictionary[id] for id in doc if id >= 0])
idx_corpus = temp_corpus

#### Dimension Reduction

In [6]:
average_doc_size = 0
for doc in idx_corpus:
    average_doc_size += len(doc)
average_doc_size /= len(idx_corpus)
average_doc_size = math.ceil(average_doc_size)

df = dictionary.dfs
filtered_corpus = []
for doc in idx_corpus:
    new_doc = [(word, df[dictionary.token2id[word]]) for word in doc]
    new_doc.sort(reverse=True, key=lambda x: x[1])
    new_doc = new_doc[:average_doc_size]
    filtered_corpus.append([word for word, df in new_doc])

### Clustering

In [7]:
def get_onehot(corpus, weight):
    dictionary = Dictionary(corpus)
#     dictionary.filter_extremes(no_below=2, no_above=1, keep_n=len(dictionary))

    bow_corpus = [dictionary.doc2bow(doc) for doc in corpus]
    if weight == 'normal':
        weight_corpus = bow_corpus
    elif weight == 'tfidf':
        tfidf = TfidfModel(bow_corpus, smartirs='ltc')
        weight_corpus = [tfidf[doc] for doc in bow_corpus]

    unique_words = [dictionary[id] for id in range(len(dictionary))]
    array = numpy.zeros((len(corpus), len(unique_words)), dtype=float)
    for i, doc in enumerate(weight_corpus):
        for id, score in doc:
            array[i, id] = score

        if weight == 'normal' and len(doc) != 0:
#             array[i] = numpy.divide(array[i], len(idx_corpus[i]))
            array[i] = numpy.divide(array[i], len(doc))
    
    return pandas.DataFrame(array, columns=unique_words, dtype=float)

In [8]:
def generate_result(predicted_labels, marks):
    result = pandas.DataFrame()
    result['comment'] = corpus
    result['tokenized_comment'] = idx_corpus
    result['predicted_label'] = predicted_labels
    if marks:
        result['marks'] = marks
    else:
        result['marks'] = -1
    return result

In [9]:
def eval_cluster(onehot_corpus, result):
    num_cluster = numpy.unique(result['predicted_label'])[-1] + 1

    clusters = [[] for i in range(num_cluster)]
    corpus_centroid = []
    for i, label in result['predicted_label'].iteritems():
        clusters[label].append(numpy.array(onehot_corpus.iloc[i]))
        corpus_centroid.append(numpy.array(onehot_corpus.iloc[i]))
    corpus_centroid = numpy.mean(corpus_centroid, axis=0).reshape(1, -1)   

#     print('\tIntra cluster sim\tInter cluster sim\tIntra / Inter')
    compactness = 0
    centroids = []
    for i in range(num_cluster):
        size = len(clusters[i])
        if size != 0:
            centroid = numpy.mean(clusters[i], axis=0)
            centroids.append(centroid)
            centroid = centroid.reshape(1, -1)
            similarities = cosine_similarity(centroid, clusters[i])
            compactness += numpy.sum(similarities)

#             intra = numpy.sum(similarities) / size
#             inter = cosine_similarity(centroid, corpus_centroid)[0][0]
#             print(i, end='\t')
#             print(intra, end='\t')
#             print(inter, end='\t')
#             print(intra / inter)
    return compactness, centroids

In [10]:
min_samples = 7
eps = 0.21
expand_rate = 0.05
epoch = 15

onehot_corpus = get_onehot(idx_corpus, 'normal')

In [ ]:
max_compactness = 0
for i in range(epoch):
    model = NewSDC()
    _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)

#     model = SDC()
#     _tpredicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate)
    
#     marks = None
    
#     model = DBSCAN(metric='cosine', eps=eps, min_samples=min_samples).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_ + 1

#     model = KMeans(n_clusters=14).fit(onehot_corpus)
#     _tpredicted_labels = model.labels_
    
    _tresult = generate_result(_tpredicted_labels, marks)
    compactness, _tcentroids = eval_cluster(onehot_corpus, _tresult)
    
    if compactness > max_compactness:
        max_compactness = compactness
        predicted_labels = _tpredicted_labels
        result = _tresult
        centroids = _tcentroids
        
print(max_compactness)
label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
num_cluster = len(label_count)
print(label_count)

#### Iterative New SDC

In [11]:
centroids = None 
prev_label_count = None
while True:
    model = NewSDC()
    predicted_labels, marks = model.predict(onehot_corpus, min_samples, eps, expand_rate, seeds=centroids)
    
    result = generate_result(predicted_labels, marks)
    compactness, centroids = eval_cluster(onehot_corpus, result)
    
    label_count = numpy.unique(result['predicted_label'], return_counts=True)[1]
    if numpy.array_equal(label_count, prev_label_count):
        break
    prev_label_count = label_count
    centroids = centroids[1:]
    
    print(compactness)
    print(label_count)
num_cluster = len(label_count)

95.36680683805503
[79 22 26 14  9 10  8 12 18 17 16 25 12]
93.26259874348283
[83 24 69 10 11  7 10 15 13 15 11]
92.55370830826335
[85 22 73 10 10  7 10 15 13 13 10]
92.58340720773641
[86 21 73 10 10  7 10 15 13 13 10]


#### Grouping

In [12]:
sims = cosine_similarity(centroids)
new_labels = [i for i in range(num_cluster)]
print(new_labels)
for i, row in reversed(list(enumerate(sims))):
    for j, value in reversed(list(enumerate(row[:i + 1]))):
        if i != j and value >= eps - eps / 20:
            print(i, j, value)
            base = min(new_labels[i], new_labels[j])
            new_labels[j] = base
            new_labels = [base if label == new_labels[i] else label for label in new_labels]
print(new_labels)

grouped_labels = numpy.zeros(len(corpus))
for i, label in enumerate(predicted_labels):
    grouped_labels[i] = new_labels[label]
new_result = generate_result(grouped_labels, None)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
9 7 0.2515612242896344
9 2 0.2304550091367859
9 0 0.2719709043191294
8 7 0.2004063447529936
[0, 1, 0, 3, 4, 5, 6, 0, 0, 0, 10]


### Result

In [13]:
class Widget:
    def __init__(self, result, column_name):
        self.result = result
        self.column_name = column_name
        
        label_count = numpy.unique(result['predicted_label'])
        self.widget = widgets.ToggleButtons(
            options=[int(num) for num in label_count],
            disabled=False,
            button_style='',
        )
        
        self.widget.observe(self.on_click, names='index')
        self.on_click({'new' : 0})
        
    def on_click(self, change):
        clear_output()
        display(self.widget)
        new = self.widget.options[change['new']]
        for index, value in self.result[self.result['predicted_label'] == new].iterrows():
            if value['marks'] == 0:
                print("@", end="")
            elif value['marks'] == 1:
                print("*", end="")
            print(index, value[self.column_name])

In [14]:
# result.to_csv('../data/results/filtered/' + file_name + '.csv', index=False)

# result = pandas.read_csv('../data/results/filtered/' + file_name + '.csv')

print(eval_cluster(onehot_corpus, result)[0])
print(numpy.unique(result['predicted_label'], return_counts=True)[1])

92.58340720773641
[86 21 73 10 10  7 10 15 13 13 10]


In [15]:
w1 = Widget(new_result, 'comment')

ToggleButtons(options=(0, 1, 3, 4, 5, 6, 10), value=0)

0 โลตัสไฮเปอร์มาเก็ต​ ทุกสาขา​ ควรจ้างพนักงานเยอะกว่านี้​ ไม่ควรเรียกพนักงานแผนกอื่นไปลงเครื่องแคชเชียร์​บ่อยจนเกินไป​ ทุกคนต้องมีหน้าที่ที่ตัวเองต้องรับผิดชอบ​ พอช่วยแคชเชียร์เสร็จ​ งานแผนกตัวเองเละก็โดนด่า​ พอโดนด่าก็เกิดความเครียด​ หน้าบึ้งตึงใส่ลูกค้า​ ทำงานไม่มีความสุข​ บริการลูกค้าไม่ดี​ แล้วลูกค้าก็ร้องเรียน​ เพราะรักจึงอยากแนะนำ​ จากใจเด็กโลตัสเก่า​
1 บางครั้งพนักงานโลตัสเอ็กเพลส กับพนักงานโลตัสใหญ่ ก็ต่างกันครับ โลตัสใหญ่ เขาทำเป็นเเผนกๆไปครับ เเต่โลตัสเอ็กเพลสนั้น พนักงานเเต่ละคน ต้องทำทุกอย่างครับ ไม่ใช่เเค่ยืนขายของ อยากให้ทุกคนเข้าใจด้วยครับ บางคนเวลากินข้าวเเทบไม่มี ผมเชื่อว่าทุกคนก็เป็นพนักงานเงินเดือนเหมือนกัน ควรเข้าใจกันบ้างครับ พนักงานคนอื่นเป็นเเบบไหนผมไม่รู้ เเต่มันก็ยังมีพนักงานที่ตั้งใจทำงานครับ ขอให้ทุกคนเข้าใจครับ
2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
4 ปรับปรุงพนักงานโลตัสเอ็กซ์เพลสสาขาตลาดนานาเจริญ ห่วยแตก ไม่เต็มใจทำก็ออกไปทำงานอย่างอื่น คนอื่นที่เข้าอยากทำงานจะได้เข้ามา ออกไป๊
7 ..เคยไปทำ เอ้กซ์เพรส เดือนเดียว พอเลย  พนักงานต่อ1ช่วงก

In [16]:
w2 = Widget(result, 'comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=0)

2 เลิกเปิดเพลง ข้าวแสนดี กับอีเครื่องกรองน้ำเพียว ได้แล้ว!!!!
8 อย่าบังคับน้องเข้าประชุมเชียร์ อย่าลงโทษโดยเหตุผลงี่เง่าๆ เกิดก่อนไม่กี่ปีเอง
10 ทุกคนล้วนแล้ว แต่มีหน้าที่ของเขา คนเราควรให้โอกาสซึ่งกันและกันนะครับให้คิดว่าคนดีก็มี คนไม่ดีก็มีนะครับ ผมไม่ได้โลกสวยนะครับ แต่ผมก็ทำงานในด้านบริการเหมือนกัน ถ้าไม่มายืนตรงจุดที่เขายืนคุณก็จะไม่รู้ คิดบวกนะครับ พนักงานเหมือนกันนะครับ ปรับปรุงตรงไหนแล้วดี ก็ควรที่ต้องปรับปรุงแก้ไขส่วนที่ผิดพลาดของตัวเรา ดีแล้วยังดีได้อีกนะครับ Mc.Tom
15 มารยาทพนักงาน เซเว่นยังคุณภาพดีกว่าหน่อยนึง แบบว่าเหี้ยทั้งคู่เลย
22 โลตัสเอ็กซ์เพรสสาขาหน้า รพ.สรรพสิทธิประสงค์ จ.อุบลราชธานี และสาขาใหญ่ถนนชยางกูร ไม่ว่าจะแผนกใด ๆ  สีหน้า ท่าทาง การบริการ เซอร์วิสมายด์ ไม่มี !!!!!!!  อยากคอมเพลนหลายครั้งแล้วค่ะ ที่ทำได้ก็แค่ปล่อยผ่าน จนตอนนี้มันทำให้เราไม่อยากเข้าไปซื้อของที่ห้างนี้เลย ..
23 เช็คเอาท์ ... หน้าจะงอไปไหนสวัสดีค่ะมีอะไรให้ช่วยมั้ยคะโอกาสหน้าเชิญใหม่ค่ะมีคลับการ์ดมั้ยคะแค่เนี้ยะ ... จะตายมั้ย
29 โลตลาด พนง.ปากตลาด นินทาลค.เผาขน ตะโกนโหวกเหวกข้ามหัวลค. วันหวยออกส

In [17]:
w3 = Widget(result, 'tokenized_comment')

ToggleButtons(options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10), value=0)

2 ['เลิก', 'เพ', 'ข้าว', 'ดี', 'อี', 'เครื่อง', 'กรอง', 'น้ํา', 'เพ', 'ียว']
8 ['อย่า', 'บังคับ', 'น้อง', 'ประช', 'อย่า', 'งี่เง่า', 'กี่', 'ปี']
10 ['้วน', 'มีหน้าที่', 'คน', 'โอกาส', 'นะครับ', 'คน', 'คน', 'นะครับ', 'ผม', 'ไม่ได้', 'โลก', 'นะครับ', 'ผม', 'ทํางาน', 'ด้านบริการ', 'เหมือนกัน', 'ยืน', 'ตรงจุด', 'ยืน', 'ไม่รู้', 'นะครับ', 'พนักงาน', 'เหมือนกัน', 'นะครับ', 'ปรับปรุง', 'ตรงไหน', 'ดี', 'ก็ควร', 'ต้องปรับปรุง', 'ตัว', 'ดี', 'แล้วยัง', 'ดี', 'นะครับ']
15 ['มารยาทพนักงาน', 'เซเว่น', 'คุณภาพ', 'ดีกว่า', 'นึง', 'เหี้ย', 'คู่']
22 ['โลตัสเอ็กซ์เพรส', 'สาขา', 'หน้า', 'สิทธิ', 'ประ', 'งค์', 'อุ', 'ราช', 'และสาขาใหญ่', 'ถนน', 'ยาง', 'แผนก', 'ใด', 'สีหน้า', 'ท่าทาง', 'บริการ', 'เซอร์วิสมายด์', 'ไม่มี', 'คอมเพ', 'หลายครั้ง', 'ปล่อย', 'ตอนนี้', 'มันทําให้', 'ไม่อยาก', 'เข้าไป', 'ซื้อของ', 'ห้าง']
23 ['เช็ค', 'หน้า', 'ไปไหน', 'สวัสดี', 'มั้ยคะ', 'โอกาส', 'หน้า', 'คลับการ์ด', 'มั้ยคะ', 'ตาย']
29 ['โล', 'ตลาด', 'พนง', 'ปาก', 'ตลาด', 'นินทา', 'ตะโกน', 'หว', 'หว', 'ข้ามหัวล', 'หว', 'ั่น', 'พิ

In [18]:
seed = 0
compare = 0

a = numpy.array(onehot_corpus.iloc[seed]).reshape(1, -1)
b = numpy.array(onehot_corpus.iloc[compare]).reshape(1, -1)
print(cosine_similarity(a,b))

print(idx_corpus[seed])
print(corpus[seed])
print(idx_corpus[compare])
print(corpus[compare])

[[1.]]
['โลตัส', 'เปอร์', 'มาเก็ต', 'ทุกสาขา', 'พนักงาน', 'เยอะกว่านี้', 'พนักงาน', 'แผนก', 'เครื่อง', 'แคชเชียร์', 'มีหน้าที่', 'ตัวเอง', 'ชอบ', 'แคชเชียร์', 'งาน', 'แผนก', 'ตัวเอง', 'เครียด', 'หน้าบึ้ง', 'ตึง', 'ใส่', 'ลูกค้า', 'ทํางาน', 'ไม่มี', 'บริการลูกค้า', 'ดี', 'ลูกค้า', '้อ', 'งเรียน', 'ัก', 'แนะนํา', 'ใจ', 'เด็ก', 'โลตัส', 'เก', '่า']
โลตัสไฮเปอร์มาเก็ต​ ทุกสาขา​ ควรจ้างพนักงานเยอะกว่านี้​ ไม่ควรเรียกพนักงานแผนกอื่นไปลงเครื่องแคชเชียร์​บ่อยจนเกินไป​ ทุกคนต้องมีหน้าที่ที่ตัวเองต้องรับผิดชอบ​ พอช่วยแคชเชียร์เสร็จ​ งานแผนกตัวเองเละก็โดนด่า​ พอโดนด่าก็เกิดความเครียด​ หน้าบึ้งตึงใส่ลูกค้า​ ทำงานไม่มีความสุข​ บริการลูกค้าไม่ดี​ แล้วลูกค้าก็ร้องเรียน​ เพราะรักจึงอยากแนะนำ​ จากใจเด็กโลตัสเก่า​
['โลตัส', 'เปอร์', 'มาเก็ต', 'ทุกสาขา', 'พนักงาน', 'เยอะกว่านี้', 'พนักงาน', 'แผนก', 'เครื่อง', 'แคชเชียร์', 'มีหน้าที่', 'ตัวเอง', 'ชอบ', 'แคชเชียร์', 'งาน', 'แผนก', 'ตัวเอง', 'เครียด', 'หน้าบึ้ง', 'ตึง', 'ใส่', 'ลูกค้า', 'ทํางาน', 'ไม่มี', 'บริการลูกค้า', 'ดี', 'ลูกค้า', '้อ', 'งเรียน', 'ัก'